In [1]:
import pandas as pd
import numpy as np
import csv
#import glob as glob
import os
from datetime import datetime, timedelta
from dateutil.parser import parse

In [2]:
import sys
import os
import re
import fnmatch
try:
    _unicode = unicode
except NameError:
    # If Python is built without Unicode support, the unicode type
    # will not exist. Fake one.
    class _unicode(object):
        pass
__all__ = ["glob", "iglob"]
def glob(pathname):
    """Return a list of paths matching a pathname pattern.

    The pattern may contain simple shell-style wildcards a la
    fnmatch. However, unlike fnmatch, filenames starting with a
    dot are special cases that are not matched by '*' and '?'
    patterns.

    """
    return list(iglob(pathname))
def iglob(pathname):
    """Return an iterator which yields the paths matching a pathname pattern.

    The pattern may contain simple shell-style wildcards a la
    fnmatch. However, unlike fnmatch, filenames starting with a
    dot are special cases that are not matched by '*' and '?'
    patterns.

    """
    dirname, basename = os.path.split(pathname)
    if not has_magic(pathname):
        if basename:
            if os.path.lexists(pathname):
                yield pathname
        else:
            # Patterns ending with a slash should match only directories
            if os.path.isdir(dirname):
                yield pathname
        return
    if not dirname:
        for name in glob1(os.curdir, basename):
            yield name
        return
    # `os.path.split()` returns the argument itself as a dirname if it is a
    # drive or UNC path.  Prevent an infinite recursion if a drive or UNC path
    # contains magic characters (i.e. r'\\?\C:').
    if dirname != pathname and has_magic(dirname):
        dirs = iglob(dirname)
    else:
        dirs = [dirname]
    if has_magic(basename):
        glob_in_dir = glob1
    else:
        glob_in_dir = glob0
    for dirname in dirs:
        for name in glob_in_dir(dirname, basename):
            yield os.path.join(dirname, name)

# These 2 helper functions non-recursively glob inside a literal directory.
# They return a list of basenames. `glob1` accepts a pattern while `glob0`
# takes a literal basename (so it only has to check for its existence).

def glob1(dirname, pattern):
    if not dirname:
        dirname = os.curdir
    if isinstance(pattern, _unicode) and not isinstance(dirname, unicode):
        dirname = unicode(dirname, sys.getfilesystemencoding() or
                                   sys.getdefaultencoding())
    try:
        names = os.listdir(dirname)
    except os.error:
        return []
    if pattern[0] != '.':
        names = filter(lambda x: x[0] != '.', names)
    return fnmatch.filter(names, pattern)

def glob0(dirname, basename):
    if basename == '':
        # `os.path.split()` returns an empty basename for paths ending with a
        # directory separator.  'q*x/' should match only directories.
        if os.path.isdir(dirname):
            return [basename]
    else:
        if os.path.lexists(os.path.join(dirname, basename)):
            return [basename]
    return []


magic_check = re.compile('[*?[]')

def has_magic(s):
    return magic_check.search(s) is not None

In [3]:
directory='/Users/kaitlynlieschke/Documents/odowd/data/*'
folders=glob(directory)

In [4]:
allfiles=list()
for folder in folders:
    files=glob(os.path.join(folder,"*.csv"))
    allfiles=allfiles+files

In [5]:
#Create headernames for all columns in file
headernames=["BMP temperature","pressure","SHT temperature","RH","dew point temp","MiCS O3","MiCS CO","MiCS NO2",
             "Alpha O3","Alpha O3","Alpha CO","Alpha CO","Alpha NO","Alpha NO","Alpha NO2","Alpha NO2","high PM",
             "lowPM","%FS PM","CO2","GMP temperature","UTC time"]

In [6]:
df=pd.concat((pd.read_csv(file,names=headernames,na_values='-999') for file in allfiles), ignore_index=True)
#,converters={'UTC time': lambda x: str(x)} <-- in pd.read_csv??

In [7]:
df.tail()

,BMP temperature,pressure,SHT temperature,RH,dew point temp,MiCS O3,MiCS CO,MiCS NO2,Alpha O3,Alpha O3.1,...,Alpha NO,Alpha NO.1,Alpha NO2,Alpha NO2.1,high PM,lowPM,%FS PM,CO2,GMP temperature,UTC time
4958808,30.66,1003.95,30.44,29.18,10.49,0.44,1.51,3.12,3.60,2.99,...,1.88,0.0,2.82,2.16,364432.0,364432.0,0.00,392.6,34.8,2015-06-04 21:45:36
4958809,30.64,1003.85,30.45,29.14,10.48,0.45,1.53,3.12,3.58,2.98,...,1.88,0.0,2.81,2.16,364541.0,364541.0,0.00,387.4,34.9,2015-06-04 21:45:41
4958810,30.62,1003.87,30.43,29.14,10.46,0.45,1.56,3.08,3.57,2.97,...,1.88,0.0,2.81,2.16,346964.0,364654.0,4.85,393.4,34.9,2015-06-04 21:45:46
4958811,30.62,1003.87,30.43,29.11,10.45,0.45,1.52,3.11,3.57,2.97,...,1.67,0.0,2.81,2.14,350834.0,365780.0,4.09,390.6,34.8,2015-06-04 21:45:51
4958812,30.59,1003.84,30.41,29.11,10.43,0.44,1.52,3.08,3.56,2.96,...,1.95,0.0,2.81,2.12,364423.0,364423.0,0.00,386.0,34.9,2015-06-04 21:45:56


Save df as only the two columns I'm interested in

In [8]:
df=df[['%FS PM','UTC time']] 

Remove all rows containing NaN (limit to two columns first so only rows with NaN in those columns are removed)

In [9]:
df=df.dropna()

Select date range

In [10]:
df=df[(df['UTC time'] >= '2014-08-23') & (df['UTC time'] <= '2015-05-31')]

Reset index for cut dataframe

In [11]:
df=df.reset_index()
df=df[['%FS PM','UTC time']] 

In [12]:
df.head()

,%FS PM,UTC time
0,3.99,2014-08-23 00:00:03
1,4.38,2014-08-23 00:00:08
2,0.00,2014-08-23 00:00:13
3,0.00,2014-08-23 00:00:18
4,0.00,2014-08-23 00:00:23


Identify any problem rows in parse()

l=0
for x in df['UTC time']:
    parse(x)
    l=l+1
print(l)
    

print(l)

Save identified problem row and attempt to parse it

c=df.loc[2373188]['UTC time']

Check another random input to parse()
c=df.loc[273188]['UTC time']

parse(c)

df=df.drop(df.index[2373188])

Create datetime column in df from the 'UTC time' strings for each row

In [13]:
df['UTC datetime']=df['UTC time'].apply(lambda row: parse(row)) 

Another check for datetime issues:
random= pd.to_datetime(df['UTC time'], dayfirst=False, errors='coerce')
print(random)

idx = random[random.isnull()].index
print(idx)
df.loc[4044928]

Convert UTC time to local time

In [ ]:
# from datetime import *
# from dateutil.tz import *
# for xx in df['UTC datetime']:
#     from_zone = tz.gettz(df['UTC datetime'][:xx])
#     to_zone = tz.tzlocal()
#     utc = datetime.strptime(df['UTC datetime'][:xx], '%Y-%m-%d %H:%M:%S')  # <====== STRING
#     utc = utc.replace(tzinfo = from_zone)
#     central = utc.astimezone(to_zone)
# # # 'central' prints utc time plus offset, not local time. How do I change that?

Create hourly average of measurements and recreate datetime column

In [14]:
df['day'] = df['datetime'].apply(lambda x: x.day)
df['month'] = df['datetime'].apply(lambda x: x.month)
df['hour']=df['datetime'].apply(lambda x: x.hour)
df['year']=df['datetime'].apply(lambda x: x.year)
df = df.drop('datetime', axis=1)
df=df.apply(lambda x: pd.to_numeric(x, errors='coerce'))
df_t = df.groupby(['day','month','hour','year'], as_index=False,sort=False)
aggregation = {'%FS PM':'mean'
                 }
df_t = df_t.agg(aggregation)
df_t['datetime'] = df_t[['year', 'month', 'day', 'hour']].apply(lambda s : datetime(*s),axis = 1)

# Plotting

In [15]:
import plotly.plotly as py
py.sign_in('kaitlynlieschke', '4gs9IuDiESfdk5Cj9POi')
import plotly.graph_objs as go

x=df_t['datetime']
y=df_t['%FS PM']

# Create trace
trace = go.Scatter(
   x = x,
   y = y,
   name = 'ODOWD'
)

data = [trace]

layout = go.Layout(
   title='% Full Scale PM',
   yaxis=dict(
       title='% FS',
       titlefont=dict(
           family='Courier New, monospace',
           size=18,
           color='#7f7f7f'
       )
   )
)


fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='odowd %FSPM')